<a href="https://colab.research.google.com/github/luismiguelcasadodiaz/IBM_SkillsBuild_IA_325/blob/main/IA_325_py_cod_ex_38_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de calidad del aire
## Contexto:

Imagina que trabajas en una empresa de tecnología verde. Te piden crear un modelo que, basándose en datos de calidad del aire, prediga si un área es saludable o contaminada para las personas.



## Objetivo:

Usando NumPy para generar datos simulados y SVM para clasificar, debes construir un modelo que prediga si un área tiene buena o mala calidad del aire.



## Requisitos:

+ #### Crear una clase AirSample.
  + Atributos:

    + pm25 (partículas finas PM2.5 en µg/m³)

    + pm10 (partículas gruesas PM10 en µg/m³)

    + o3 (ozono en ppb)

    + no2 (dióxido de nitrógeno en ppb)

    + quality (0 = saludable, 1 = contaminado) → solo en entrenamiento.

+ #### Crear una clase AirDataGenerator para generar datos sintéticos:

  Regla orientativa: si pm25 > 35 o pm10 > 50 o no2 > 40 → contaminado.

+ #### Crear una clase AirQualityClassifier para:

  Entrenar un modelo SVM usando los datos generados.

  Predecir la calidad del aire de una nueva muestra.

+ #### Crear una clase AirQualityExample para:

  + Generar muestras.

  + Entrenar el clasificador.

  + Predecir la calidad de una muestra nueva (creada manualmente).

  + Mostrar el resultado de la predicción.



## Ejemplo de uso
```python
example = AirQualityExample()
example.run()
```
## Salida esperada
```python
🌍 Muestra de aire:
PM2.5: 22, PM10: 30, O3: 50, NO2: 35
✅ Predicción de calidad: Saludable ✅
```

## Importación de librerías

In [63]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import numpy as np

## Definición de la clase AirSample

In [64]:
class AirSample:
  """
  Representa una muestra de aire con diferentes métricas de calidad.

  Atributos:
      pm25 (float): Concentración de partículas finas PM2.5 en µg/m³.
      pm10 (float): Concentración de partículas gruesas PM10 en µg/m³.
      o3 (float): Concentración de ozono en ppb.
      no2 (float): Concentración de dióxido de nitrógeno en ppb.
      q (int, opcional): Calidad del aire (0 = saludable, 1 = contaminado).
                         Por defecto es None.
  """
  def __init__(self, pm25, pm10, o3, no2, q=None):
    """
    Inicializa una nueva instancia de AirSample.

    Args:
        pm25 (float): Concentración de partículas finas PM2.5.
        pm10 (float): Concentración de partículas gruesas PM10.
        o3 (float): Concentración de ozono.
        no2 (float): Concentración de dióxido de nitrógeno.
        q (int, opcional): Calidad del aire. Por defecto es None.
    """
    self.pm25 = pm25
    self.pm10 = pm10
    self.o3 = o3
    self.no2 = no2
    self.q = q
  def __str__(self):
    return f"PM2.5: {self.pm25}, PM10: {self.pm10}, O3: {self.o3}, NO2: {self.no2}"
  def to_vector(self):
    """
    Convierte los atributos de la muestra (sin la calidad) en un vector numérico.

    Este método es útil para preparar los datos para el entrenamiento de modelos
    de aprendizaje automático.

    Returns:
        list: Una lista de floats con las métricas de la muestra
              [pm25, pm10, o3, no2].
    """
    return [self.pm25, self.pm10, self.o3, self.no2]

## Definición de la clase AirDataGenerator

In [65]:
class AirDataGenerator:
  """
  Generador de datos sintéticos para muestras de calidad del aire.

  Atributos:
      num_samples (int): El número de muestras a generar.
      samples (list): Una lista para almacenar las muestras generadas.
  """
  def __init__(self, num_samples):
    self.num_samples = num_samples
    self.samples = []
  def generate(self):
    """
    Genera un número especificado de muestras de aire sintéticas.

    La calidad (contaminado o saludable) se determina basándose en umbrales
    simples para PM2.5, PM10 y NO2.

    Returns:
        list: Una lista de objetos AirSample generados.
    """
    for _ in range(self.num_samples):
      pm25 = np.random.uniform(0, 150)
      pm10 = np.random.uniform(0, 200)
      o3 = np.random.uniform(0, 120)
      no2 = np.random.uniform(0, 60)
      polluted = int(pm25 > 50 or pm10 > 80 or o3 > 100  or no2 > 40)
      sample = AirSample(pm25, pm10, o3, no2, polluted)
      self.samples.append(sample)
    return self.samples


## Definición de la clase AirQualityClassifier

In [66]:
class AirQualityClassifier:
  """
  Clasificador de calidad del aire basado en un modelo SVM.

  Permite entrenar un modelo para predecir si una muestra de aire es
  saludable o contaminada.

  Atributos:
      model: Instancia del modelo SVC de scikit-learn, inicialmente None.
  """
  def __init__(self):
    self.model = None
  def fit(self,samples):
    """
    Entrena el modelo SVM utilizando una lista de objetos AirSample.

    Extrae las características y etiquetas de las muestras para entrenar
    el clasificador. Si el modelo ya está entrenado, imprime un mensaje.

    Args:
        samples (list): Una lista de objetos AirSample con etiquetas de calidad (q).
    """
    if self.model is None:
      X = [sample.to_vector() for sample in samples]
      y = [sample.q for sample in samples]
      self.model = SVC(kernel='linear')
      self.model.fit(X,y)
    else:
      print("El modelo ya está entrenado")
  def predict(self,sample):
    """
    Predice la calidad del aire para una sola muestra de AirSample.

    Si el modelo no está entrenado, imprime un mensaje y retorna None.

    Args:
        sample (AirSample): Un objeto AirSample para predecir su calidad.

    Returns:
        int or None: La predicción de calidad (0 = saludable, 1 = contaminado)
                     o None si el modelo no está entrenado.
    """
    if self.model is None:
      print("El modelo no está entrenado")
      return None
    else:
      X = [sample.to_vector()]
      y_pred = self.model.predict(X)
      return y_pred[0]



## Definición de la clase AirQUalityExample

In [67]:
class AirQualityExample:
  def __init__(self):
    self.generator = AirDataGenerator(1000)
    self.classifier = AirQualityClassifier()
  def run(self):
    samples = self.generator.generate()
    self.classifier.fit(samples)
    new_sample = AirSample(22, 30, 50, 35)
    prediction = self.classifier.predict(new_sample)
    print("🌍 Muestra de aire:")
    print(new_sample)
    if prediction == 0:
      print("✅ Predicción de calidad: Saludable ✅")
    else:
      print("❌ Predicción de calidad: Contaminado ❌")

In [68]:
example = AirQualityExample()
example.run()

🌍 Muestra de aire:
PM2.5: 22, PM10: 30, O3: 50, NO2: 35
✅ Predicción de calidad: Saludable ✅
